## prompt, model, output_parser

In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Why don\'t ice cream cones ever go to school? \n\nBecause they already have a lot of "scoops" of knowledge!'

## RAG

In [67]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

## Paralelism

In [68]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Sure, here's a bear joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"),
 'poem': AIMessage(content='In the shadows he roams, a wild mystique,\nA gentle giant, fierce yet meek.')}

In [70]:
joke_chain.batch([{"topic":"bear"}, {"topic": "horse"}])

[AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!"),
 AIMessage(content='Why did the horse sit on the fence?\n\nBecause it wanted to be a little "stable"!')]

## SQL DB

In [71]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [72]:
from langchain_community.utilities import SQLDatabase

will not work

In [74]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")

In [75]:
def get_schema(_):
    return db.get_table_info()

In [76]:
def run_query(query):
    return db.run(query)

In [77]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [81]:
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) as number_of_employees\nFROM employees'

In [78]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [79]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

will not work, db not set up

In [80]:
full_chain.invoke({"question": "How many employees are there?"})

OperationalError: (sqlite3.OperationalError) no such table: employees
[SQL: SELECT COUNT(*) FROM employees;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Code

In [82]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

In [83]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI()

In [84]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

In [89]:
chain = prompt | model | StrOutputParser() # | _sanitize_output | PythonREPL().run

chain.invoke({"input": "give code to calculate 2+2 and print it out"})

'```python\nresult = 2 + 2\nprint(result)\n```'

## tools

In [92]:
!pip install --quiet duckduckgo-search

In [93]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [94]:
search = DuckDuckGoSearchRun()

In [95]:
template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [96]:
chain = prompt | model | StrOutputParser() | search

In [97]:
chain.invoke({"input": "I'd like to figure out what games are tonight"})

/home/victor/.local/lib/python3.8/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


'How to live stream NBA games tonight. Pelicans vs Thunder and Clippers vs Lakers will air on ESPN. Viewers can also stream NBA games on Sling TV. Fans in the U.S. can watch the biggest games of ... Among the best NBA games tonight, we\'re highlighting the Memphis Grizlies vs Boston Celtics (6:00 PM ET), the Toronto Raptors vs Oklahoma City Thunder (7:00 PM ET) and the Milwaukee Bucks vs ... Here is everything you need to know about the "Monday Night Football" doubleheader, including TV and streaming options for each game. What\'s better than a "Monday Night Football" doubleheader? A second "Monday Night Football" doubleheader.. ESPN is bringing fans two prime-time games again this week after its doubleheader in Week 2. Here\'s all you need to know to watch the NFL\'s Monday night\'s doubleheader in Week 2.'